In [ ]:
# sc.list_packages()
sc.install_pypi_package("pandas==0.25.1")
sc.install_pypi_package("boto3==1.26.7")
sc.install_pypi_package("rapidfuzz")

: 

In [2]:
confs = ["spark.yarn.executor.memoryOverhead","spark.sql.shuffle.partitions","spark.network.timeout","spark.driver.memory"]
print(sc._conf.get('spark.app.name'))
print(sc._conf.get('spark.submit.deployMode'))
print(sc._conf.get("spark.yarn.executor.memoryOverhead"))
print(sc._conf.get("spark.sql.files.maxPartitionBytes"))
for conf in confs:
    print(conf,":",sc._conf.get(conf))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

livy-session-1
client
None
None
spark.yarn.executor.memoryOverhead : None
spark.sql.shuffle.partitions : None
spark.network.timeout : None
spark.driver.memory : 2048M

# Imports

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth
import boto3
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, MapType, IntegerType, DoubleType
from pyspark.sql.types import *
from pyspark.sql import functions as F
from collections import Counter
import pyspark
import rapidfuzz
from rapidfuzz import fuzz
import time
import random

# import s3fs
# import boto3
#How to pip install while creation of cluster

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:

#          .config("spark.memory.offHeap.enabled","true") \
#          .config("spark.memory.offHeap.size","10g") \
# .config("spark.submit.deployMode","client") \
#         .config("spark.driver.memory","200g") \
#         .config("spark.executor.memory","50g") \
#         .config("spark.emr.default.executor.memory", '50g') \

# spark.conf.set("spark.sql.shuffle.partitions",100)

#Number of partitions should be equal to or greater than the cores to achieve parallelism
# config = pyspark.SparkConf().setAll([('spark.executor.memory', '80'), ('spark.executor.cores', '12'), \
#                                      ('spark.cores.max', '240'), ('spark.driver.memory','300g'), ('spark.sql.shuffle.partitions','80')])
# spark.sparkContext.stop()
# config = pyspark.SparkConf().setAll([('spark.network.timeout', '100800s'), ('spark.driver.memory','122000M')]) #, ('spark.executor.cores', '12')])
# spark = SparkSession.builder.config(conf=config).appName('ProcessingData') \
#         .getOrCreate()
spark = SparkSession.builder.getOrCreate()
# sc = pyspark.SparkContext.getOrCreate(conf=config)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Task
Read data of specific countries heritage sites. Within them, if any org in any row is a close match to the NATO organization then return a one. Groupby with average tone, and count for total, NATO orgs. Groupby level, country, month.


Countries:
1. Serbia - RI
2. Libya - LY
3. Mali - ML



## 1.  Reading Files

In [5]:
from boto3 import client

def return_yearly_parquet_files(year):
    print(f"Return file list for the year: {year}")
    file_list = []
    conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
    for key in conn.list_objects(Bucket='kcsra', Prefix = f"{str(year)} GDELT GKG PG")['Contents']:
        file = "s3://kcsra/"+ key['Key']
        if "parquet" in file:
            file_list.append(file)
    return file_list

def folder_parquet_files(year):
    print(f"Return file list for the year: {year}")
    file_list = []
    conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
    for key in conn.list_objects(Bucket='kcsra', Prefix = f"{str(year)} GDELT GKG PG")['Contents']:
        file = "s3://kcsra/"+ key['Key']
        if "parquet" in file:
            file_list.append(file)
    return file_list


def return_correct_file_paths(year_file_list):
    print("Returning the working parquet files ...")
    correct_file_paths = []
    for i,file in enumerate(year_file_list):

        try:
            df = spark.read.parquet(file)
            correct_file_paths.append(file)
        except Exception as e:
            print(f"[{str(i+1)}/{len(year_file_list)}]")
            print("->ERROR while reading:")
            print(file)
            print()
    return correct_file_paths

def return_df(correct_file_paths, req_cols, grid_id = None, prio_grid_list = None):
    print("Constructing a dataframe from required file paths")

    df = spark.read.parquet(*correct_file_paths).select(*req_cols).dropDuplicates(['gkgrecordid','GRID_IDs'])
    
    if grid_id:
        print("Filter Grid ID:", grid_id)
        df = df.filter(df.GRID_IDs == grid_id)
    
    if prio_grid_list:
        print("Filter Prio Grid ID List:", prio_grid_list)
#         print(type(prio_grid_list))
        df = df.filter(df.GRID_IDs.isin(prio_grid_list))

    df = df.withColumn('day',dayofmonth(df.date)).withColumn('month',month(df.date))
    
    print("Total rows:",df.count())
    # df.printSchema()
    return df

def extract_tone(tone_string):
    try:
        tone_float = re.findall("tone=(.*?),",str(tone_string))[0].strip("'")
        tone_float = float(tone_float)
        return tone_float
    except Exception as e:
        print("Error extracting tone:" + str(e))
        print(tone_string)
        print("="*20)
        return None
    
def save_tone_in_new_col(df):
    print("Saving the tone in a new column")
    extract_tone_udf = udf(extract_tone, DoubleType())
    df = df.withColumn('Extracted_tone', extract_tone_udf(df['tone'])).drop("tone").withColumnRenamed("Extracted_tone", "tone")
    return df

@udf(returnType=ArrayType(StringType()))
def retrieve_theme_list(themes):
    if not themes:
        return {}
    themes = str(themes)
    return list(Counter(re.findall(f"theme=(.*?),",themes)).keys())

@udf(returnType=ArrayType(StringType())) 
def retrieve_organization_list(organizations):
    if not organizations:
        return {}
    organizations = str(organizations)
    return list(Counter(re.findall(f"organization=(.*?),",organizations)).keys())

@udf(returnType=StringType())
def aggregate_category(category):
    super_counter = Counter({})
    for row in category:
        super_counter += Counter(row)
    return ", ".join([f"{categ[0]}:{categ[1]}" for categ in super_counter.most_common(20)])


def are_strings_similar(target_str, data_str, match_threshold):
    target_str, data_str = str(target_str).lower(), str(data_str).lower()
    partial_match, sorted_match = fuzz.partial_ratio(target_str, data_str), fuzz.token_sort_ratio(target_str, data_str)

    if max(partial_match, sorted_match) >= match_threshold:
        # print(max(partial_match, sorted_match))
        return True
    else:
        return False


def compare_string_matches_lists(target_list, data_list, match_threshold = 75):
    for target_str in target_list:
        for data_str in data_list:
            if are_strings_similar(target_str, data_str, match_threshold):
                # print("Match Found", f"| {target_str} : {data_str}")
                return 1
    return 0


def udf_compare_strings_lists(target_list, match_threshold):
    return udf(lambda data_list: compare_string_matches_lists(target_list, data_list, match_threshold))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
target_str = "North Atlantic Treaty Organization"
data_str = "North Atlantic"
match_threshold = 78
are_strings_similar(target_str, data_str, match_threshold)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

# MAIN

In [7]:
#Collect all the orgs in a list and run the matcher function to label primary and secondary firms

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
nato_org = """NATO
North Atlantic Treaty Organization"""


nato_org_list = [nato.strip() for nato in nato_org.split("\n")]
nato_org_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['NATO', 'North Atlantic Treaty Organization']

In [9]:
# match_str = """[Google, Ballistic Research Centre, Institute Of Forensic Sciences, Torrent Pharmaceuticals, Bajaj Alliance, Directorate Of Forensic Sciences, Institute Of Behavioral Sciences, Gujarat Forensic Sciences University, Formula Group, Institute Of Research Development, Congress Mp Shashi Tharoor, Telecom Day, Us Central Intelligence Agency, Cadila Pharmaceuticals, Directorate Of Forensic Science Laboratory]"""

# data_list = [x.strip() for x in match_str.strip().strip("[").strip("]").split(",")]
# compare_string_matches_lists(prim_firm_list, data_list, match_threshold = 75)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
country_prio_map = {'RI': [191201],
 'LY': [176789, 176785, 176804, 165261, 173179],
 'ML': [149391, 153715, 150114, 153000]}

match_threshold = 91
req_cols = ['gkgrecordid','date','tone','country', 'year','GRID_IDs', 'organizations']
t0 = time.time()
for country in country_prio_map.keys():
    if country == "RI":
        pass
    else:
        continue
    for year in [2015,2016,2017,2018,2019,2020,2021,2022]:#,2020,2022]: #range(2015, 2023):
        t1 = time.time()
        print(f"Country: {country} | Year: {year}")
        prio_grid_list = country_prio_map[country]
        
#         file_list = return_yearly_parquet_files(year)
#         file_list2 = [file for file in file_list if country in file]
#         correct_file_paths = return_correct_file_paths(file_list2)
        
#         correct_file_paths = random.sample(correct_file_paths, int(len(correct_file_paths)/2))

        #--------------------------------------------------------------------------------------
        #Injecting manually generated files due to error in original files:
        correct_file_paths = [f"s3://shrivats-dev/Data/#23_PrioGridLabelled/{country}/{year}"]
#         correct_file_paths = ['s3://shrivats-dev/Data/#23_PrioGridLabelled/SF/2016/part-00000-697f4c00-5877-4831-b597-ba556d26223c-c000.snappy.parquet']
        #--------------------------------------------------------------------------------------
        
        df = return_df(correct_file_paths, req_cols, prio_grid_list = prio_grid_list)
        
        
        df = save_tone_in_new_col(df)
        df = df.dropna(subset = ["tone"])
        req_cols_grpby = ['country', 'day', 'month', 'year', 'tone', 'Nato']
        t2 = time.time()
        
        print("Time to read files:", t2-t1)
        
        df2 = df.withColumn("Org_list", retrieve_organization_list(F.col("organizations")))
        # df.unpersist()

        df3 = df2.withColumn('Nato', udf_compare_strings_lists(nato_org_list, match_threshold)(F.col("Org_list")).cast(IntegerType()))\
                .select(*req_cols_grpby)
        # df2.unpersist()
        
        df4 = df3.withColumn("NatoTone", F.col("Nato") * F.col("tone"))
                 
        
        t3 = time.time()
        print("T3 time:", t3-t2)
        # df3.unpersist()
        aggregation_cols = ['country','month','year']
        df5 = df4.groupby(*aggregation_cols).agg(F.count(F.col("tone")).alias('TotalArticles'),\
                                  F.avg('tone').alias('AverageSentiment'),\
                                  F.sum('Nato').alias('Nato_TotalArticles'),\
                                F.sum('NatoTone').alias('SumNatoTone'))
                                  
        # df4.unpersist()
        df6 = df5.withColumn("AverageSentiment_Nato", F.col("SumNatoTone")/F.col("Nato_TotalArticles"))\
            .drop("SumNatoTone").fillna(0)
        # df5.unpersist()
        t4 = time.time()
        print("T4 time:", t4-t3)
#         continue
        df6.repartition(1).write.option("header",True) \
           .csv(f"s3://shrivats-dev/Tasks/#38_NATO_Sentiment/{country}/{year}")
        df6.unpersist()
        t5 = time.time()
        print("T5 time:", t5-t4)
        print("="*50)
        
        # break
print("Total time", time.time()-t0)
        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Country: RI | Year: 2015
Constructing a dataframe from required file paths
Filter Prio Grid ID List: [191201]
Total rows: 67
Saving the tone in a new column
Time to read files: 1.2914097309112549
T3 time: 0.03980112075805664
T4 time: 0.04251432418823242
T5 time: 1.9462265968322754
Country: RI | Year: 2016
Constructing a dataframe from required file paths
Filter Prio Grid ID List: [191201]
Total rows: 63
Saving the tone in a new column
Time to read files: 1.0856688022613525
T3 time: 0.04265451431274414
T4 time: 0.03257298469543457
T5 time: 1.7622535228729248
Country: RI | Year: 2017
Constructing a dataframe from required file paths
Filter Prio Grid ID List: [191201]
Total rows: 49
Saving the tone in a new column
Time to read files: 1.0385966300964355
T3 time: 0.042794227600097656
T4 time: 0.03257632255554199
T5 time: 1.205320119857788
Country: RI | Year: 2018
Constructing a dataframe from required file paths
Filter Prio Grid ID List: [191201]
Total rows: 30
Saving the tone in a new colu

In [42]:
df4.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---+-----+----+-----------------+----+--------+
|country|day|month|year|             tone|Nato|NatoTone|
+-------+---+-----+----+-----------------+----+--------+
|     RI| 10|    3|2022|0.728155339805825|   0|     0.0|
|     RI| 16|    4|2022|-1.61920714684534|   0|    -0.0|
|     RI| 23|    6|2022|-1.20897521768252|   0|    -0.0|
|     RI| 29|    7|2022|-1.76026823134954|   0|    -0.0|
|     RI| 21|   11|2022|-4.25531914893617|   0|    -0.0|
|     RI| 31|    5|2022|-3.60248447204969|   0|    -0.0|
|     RI| 22|   11|2022|-6.58135283363803|   0|    -0.0|
|     RI|  2|   12|2022|-6.41592920353982|   0|    -0.0|
+-------+---+-----+----+-----------------+----+--------+

In [43]:
#After filtering on prio_grid:
print("DF:", df.count())
print("DF4:",df4.count())
print("Perc:", df.count()/df4.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DF: 8
DF4: 8
Perc: 1.0

In [44]:
df6.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+----+-------------+------------------+------------------+---------------------+
|country|month|year|TotalArticles|  AverageSentiment|Nato_TotalArticles|AverageSentiment_Nato|
+-------+-----+----+-------------+------------------+------------------+---------------------+
|     RI|    4|2022|            1| -1.61920714684534|                 0|                  0.0|
|     RI|    5|2022|            1| -3.60248447204969|                 0|                  0.0|
|     RI|    7|2022|            1| -1.76026823134954|                 0|                  0.0|
|     RI|    6|2022|            1| -1.20897521768252|                 0|                  0.0|
|     RI|   11|2022|            2|-5.418335991287099|                 0|                  0.0|
|     RI|   12|2022|            1| -6.41592920353982|                 0|                  0.0|
|     RI|    3|2022|            1| 0.728155339805825|                 0|                  0.0|
+-------+-----+----+-------------+----------------

In [45]:

# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
df4.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df4.columns if 'date' not in c]
   ).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---+-----+----+----+----+--------+
|country|day|month|year|tone|Nato|NatoTone|
+-------+---+-----+----+----+----+--------+
|      0|  0|    0|   0|   0|   0|       0|
+-------+---+-----+----+----+----+--------+

In [46]:
cols = ['Nato']
agg_cols = {col:'sum' for col in cols}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
df3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---+-----+----+-----------------+----+
|country|day|month|year|             tone|Nato|
+-------+---+-----+----+-----------------+----+
|     RI| 10|    3|2022|0.728155339805825|   0|
|     RI| 16|    4|2022|-1.61920714684534|   0|
|     RI| 23|    6|2022|-1.20897521768252|   0|
|     RI| 29|    7|2022|-1.76026823134954|   0|
|     RI| 21|   11|2022|-4.25531914893617|   0|
|     RI| 31|    5|2022|-3.60248447204969|   0|
|     RI| 22|   11|2022|-6.58135283363803|   0|
|     RI|  2|   12|2022|-6.41592920353982|   0|
+-------+---+-----+----+-----------------+----+

In [48]:
df3.agg(agg_cols).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|sum(Nato)|
+---------+
|        0|
+---------+

## Joining Files

### Country Level

In [51]:
folder = "#38_NATO_Sentiment"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
# countries = ["MZ", "MI", "SG","GA","SF","CM"]
countries = ["ML", "LY","RI"]
for country in countries:
    df_combined = spark.read.option("header",True) \
               .csv(f"s3://shrivats-dev/Tasks/{folder}/{country}/*/*.csv")
    df_combined.repartition(1).write.option("header",True) \
               .csv(f"s3://shrivats-dev/Tasks/{folder}/{country}/AllYears")    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### All

In [55]:
country_string = "_".join(countries)
country_string

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'ML_LY_RI'

In [56]:
df_combined_all_countries = spark.read.option("header",True) \
               .csv(f"s3://shrivats-dev/Tasks/{folder}/*/2*/*.csv")
df_combined_all_countries = df_combined_all_countries.withColumn("month", F.col("month").cast(IntegerType()))
df_combined_all_countries_sorted = df_combined_all_countries.sort("country","year","month")
df_combined_all_countries_sorted.repartition(1).write.option("header",True) \
               .csv(f"s3://shrivats-dev/Tasks/{folder}/{country_string}")  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…